## 1.5 Примеры задач решаемых при помощи методов машинного обучения


Рассмотрим примеры решения задач классификации и линейной регресии на различных типах данных.


Будем использовать библиотеки:
* numpy
* [skilearn](https://scikit-learn.org/stable/) - 'toy' датасеты, ML алгорытмы 
* [pandas](https://pandas.pydata.org/) - удобная работа с табличными данными

познакомимся с инструментами:

* **Pytorch** 
* **Tensorboard**


<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/intro-16.jpg" >

### 1.5.1 Загрузка данных

In [ ]:
# Классификация вин 
# Используем библиотеку sklearn: https://scikit-learn.org/stable/ 

import sklearn 
from sklearn.datasets import load_wine
#https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine

# Загрузка датасета
data = load_wine(return_X_y = True) # Так же можно получить данные в Bunch(словарь) или pandas DataFrame

features = data[0] # Массив 178x13 178 бутылок у каждой 13 признаков
class_labels = data[1] # Массив из 178 элементов каждый элемент это число обозначающее класс к которому относиться данная бутылка : 0,1 2  
print("Данные",features.shape)
print("Номер класса",class_labels.shape)



### 1.5.2 Визуализация данных

In [ ]:
# Подключим библиотеку для работы с табличнымии данными: https://pandas.pydata.org/
import pandas as pd 

data_bunch = load_wine(return_X_y = False)
print(data_bunch.keys())
"""
  Если параметр return_X_y == False
  Данные в объекте Bunch: https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html#sklearn.utils.Bunch 
  По сути это словарь.
  Что бы отобразить данные в виде таблицы преобразуем их в формат pandas.DataFrame
"""

df = pd.DataFrame(data_bunch.data, columns=data_bunch.feature_names)
df.head()

Каждая строка в таблице может быть интерпретированна как вектор из 13 элементов. Можно интерпретировать такой вектор как координаты точки в 13 - мерном пространстве. Именно с таким представлением работают большинство алгоритмов машинного обучения. 

Визуализировать 13-мерное пространство не получиться :(. 

Но можно визуализировать проекцию данных в 3-х мерное пространство. Для этого воспользуемся инструментом projector из tensorboard

https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

In [ ]:
# Вспомогательный метод для запуска Tensorboard в Colab

# Fix: https://stackoverflow.com/questions/60730544/tensorboard-colab-tensorflow-api-v1-io-gfile-has-no-attribute-get-filesystem
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import os
import shutil

# Запуск Tensorboard в Colab
def reinit_tensorboard(clear_log = True):
  # Лог-файлы читаются из этого каталога: 
  logs_base_dir = "runs"
  if clear_log:
    # Очистка логов
    #!rm -rfv {logs_base_dir}/*
    shutil.rmtree(logs_base_dir, ignore_errors = True)
    os.makedirs(logs_base_dir, exist_ok=True)
  # Магия Colab
  %load_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

После загрузки Tensorboard измените значение опции "Color by" на "label 3 colors" что бы объекты принадлежащие к разным классам отображались разными цветами.

from torch.utils.tensorboard import SummaryWriter
import numpy

reinit_tensorboard()
writer = SummaryWriter(comment = "wine")
np_f = numpy.array(features)
writer.add_embedding(np_f, metadata=class_labels )
writer.close()


Рассказ про PCA

Видно что объекты классов 1 и 2 линейно не разделимы в 2-х измерениях. По этой причине так популярен переход к пространствам большей размерности. 

Обратите внимание что данные центрированны около нуля - это результат нормализации которой они подверглись в Tensorboard.

Нам тоже потребуется нормализовыть данные.

## 1.5.3 Нормализация данных

<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/v2/min_max.png" >
<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/v2/norm_distr.png" >

In [ ]:
# Сделаем это средствами pytorch

import torch
from torch.utils.tensorboard import SummaryWriter

reinit_tensorboard()
writer = SummaryWriter(comment = "wine")


# Отобразим значения двух параметров значения которых отличаются примерно на порядок
f_names = data_bunch.feature_names
for i, feature in enumerate(features):
  writer.add_scalars("Raw_2_par",{ 
      f_names[1]:feature[1], # malic_acid
      f_names[3]:feature[3],  # alcalinity_of_ash
     } ) 

# Добавим еще один значения которого отличается от второго на 2 порядка

for i, feature in enumerate(features):
  writer.add_scalars("Raw_3par",{ 
                                     f_names[1]:feature[1], # malic_acid
                                     f_names[3]:feature[3],  # alcalinity_of_ash
                                     f_names[12]:feature[12] # proline 
                                     } ) 

# Добавим гистограмму для сырых данных.
writer.add_histogram("1.Raw" , features[:,3])
writer.add_histogram("1.Raw" , features[:,1])



# Преобразовали данные к torch.Tensor 
tensor_f = torch.tensor(features)

# Mini-Max  нормализация

# torch.min и torch.max возвращают кортежи (values, indexes)
# https://pytorch.org/docs/stable/generated/torch.min.html#torch.min

min_values, _  = tensor_f.min(dim=1,keepdim=True)  # shape = (178,1)
max_values, _  = tensor_f.max(dim=1,keepdim=True)  # shape = (178,1)

# Вычитаем минимальное значение
min_max_centered = tensor_f - min_values
# Делим на среднее
min_max_normalized =  min_max_centered / (max_values - min_values)

writer.add_histogram("2.Min_Max_Centered" , min_max_centered[:,3])
writer.add_histogram("2.Min_Max_Centered" , min_max_centered[:,1])

writer.add_histogram("2.Min_Max_Normalized" , min_max_normalized[:,3])
writer.add_histogram("2.Min_Max_Normalized" , min_max_normalized[:,1])

# Стандартизация / Z-нормализация

# Вычитаем среднее
centered = tensor_f - tensor_f.mean(dim=0)
# Разделим на стандартное отклонение
normalized = centered / tensor_f.std(dim=0)

# Добавим гистограмму для стандартизированных данных в Tensorboard
writer.add_histogram("3.Centered" , centered[:,3])
writer.add_histogram("3.Centered" , centered[:,1])

writer.add_histogram("3.Normalized" , normalized[:,3])
writer.add_histogram("3.Normalized" , normalized[:,1])


writer.add_histogram("4.Mix: raw, MM, Z" , features[:,1])
writer.add_histogram("4.Mix: raw, MM, Z" , min_max_normalized[:,1])
writer.add_histogram("4.Mix: raw, MM, Z" , normalized[:,1])





writer.close()

## 1. После загрузки Tensorboard выберите пункт меню "SCALARS"
затем 'Horizontal Axis' = Relative

Значения в разных масштабах - несравнимы между собой

## 2. выберите пункт меню "HISTOGRAMS"
затем Offset time axis = WALL или RELATIVE

Наглядное приемущество Стандартизации перед max-min нормализацией


### 1.5.4 Обучение

Базовая документация
https://scikit-learn.org/stable/modules/svm.html

Пример использования SVM классификатора
https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python





In [ ]:
# Split data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, class_labels, test_size=0.2) # 80% training and 20% test

print("X_train",X_train.shape)
print("X_test",X_test.shape)


##№ 1.5.5 Обучим модель и посчитаем точность (Accuracy)

In [ ]:
from sklearn import svm
from sklearn import metrics

# Создаем модель
lin_clf = svm.LinearSVC()

# Обучаем модель на части данных
lin_clf.fit(X_train, y_train)

# Получаем предсказания
y_pred = lin_clf.predict(X_test)
print("y_pred",y_pred.shape)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

#Аналогичным образом можно работать с различными типами данных

**Загрузка даннных.**

В Pytorch есть три библиотеки для работы с разными типами данных:

[torchvision](https://pytorch.org/docs/stable/torchvision/datasets.html)

[torchaudio](https://pytorch.org/audio/stable/datasets.html)

[torchtext](https://pytorch.org/text/stable/index.html)


Для загрузки данных  используются классы [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) и [Dataloader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader). 

Они предоставляют единый интерфейс для доступа к данным различных типов.



# Пример загрузки аудио средствами Pytorch

Установим библиотеку torch.audio она не входит в список пакетов доступных в colab по умолчанию.

In [ ]:
!pip install torchaudio==0.7.0

##Загрузим датасет

Speech Commands:  A Dataset for Limited-Vocabulary SpeechRecognition

https://arxiv.org/pdf/1804.03209.pdf

https://pytorch.org/audio/stable/datasets.html#speechcommands

Данные будут распакованны в папку sample_data

In [ ]:
import torchaudio
speech_commands_dataset = torchaudio.datasets.SPEECHCOMMANDS("sample_data",download = True)


Объект speech_commands_dataset - это экземпляр класса который является наследником  [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html) это означает что в нем реализованы методы 
* __getitem__ 
* __len__

Благодаря этому, мы можем узнать количество элементов или получить произвольный элемент данных обращаясь к объекту класса Dataset  так же как к обычному списку в python.

In [ ]:
print("Количество элементов {} ".format(len(speech_commands_dataset)))
print("Первый элемент",speech_commands_dataset[0])

## Что представляет из себя элемент аудио - данных?
Обратимся к документации: https://pytorch.org/audio/stable/datasets.html#speechcommands

... returns:

    (waveform, sample_rate, label, speaker_id, utterance_number)

utterance_number - номер повтора. Больше нуля если один и тот же человек проговаривает одну и ту же фразу несколько раз. 

In [ ]:
waveform, sample_rate, label, speaker_id, utterance_number = speech_commands_dataset[0]
print("Waveform: {}\nSample rate: {}\nLabel: {} \nSpeaker_id: {} \nutterance_number: {}".format(waveform.shape, sample_rate, label,speaker_id,utterance_number))


Размеры тензора waveform:
    
    [1, 16000] 

1- количество каналов, 16000 количество измерений в секунду

Если частота дискретизации(sample_rate) равна 16000 то этот фрагмент занимает ровно 1 секунду 

Визуализируем их:
x - время
y - давление

In [ ]:
import matplotlib.pyplot as plt
print(type(waveform))
plt.figure()
plt.title(f"Label: {label}")
plt.plot(waveform.t().numpy())

# Озвучим:

In [ ]:
import IPython.display as ipd
ipd.Audio(waveform.numpy(), rate=sample_rate)

## Итерация по датасету.

Для начала запустим простую проверку: убедимся что все записи одинаковой длины.

### Почему это важно

* list
* numpy - массив
* torch.tensor

Проверим что все записи имеют одинаковую длину. 

In [ ]:
import torch

def_length = 16000
for i, sample in enumerate(speech_commands_dataset):
  waveform, sample_rate, label, speaker_id, utterance_number = sample
  if def_length != waveform.shape[1]: # [1, 16000]
    print(i)
    print("Waveform: {}\nSample rate: {}\nLabel: {} \nSpeaker_id: {} \nutterance_number: {}".format(waveform.shape, sample_rate, label,speaker_id,utterance_number))
    break
  if not i% 10000 and i > 0 :
    print(f"Processed {i} objects") 


Если не все элементы будут иметь различную длину мы не сможем их сравнивать. И даже технически поместить в один массив. Необходомо их выровнять. Так как многие записи начинаются и заканчиваются тишиной, то просто дополним их нулями.
Для этого применим концепцию трансформаций (transform) которая широко применяется в Pytorch и встраивается во многие датасеты.

In [ ]:
import torchaudio

class PadWaveform(torch.nn.Module):
  def __init__(self, desired_size = 16000):
    self.desired_size = desired_size
    super().__init__()

  # in nn.Module forward method called inside __call__ method

  def forward(self, waveform):
    if waveform.shape[1] < self.desired_size:
      diff = self.desired_size - waveform.shape[1]
      pad_left = diff // 2
      pad_right = diff - pad_left
      return torch.nn.functional.pad(waveform,[pad_left, pad_right])
    else:
      return waveform

class customSpeechCommandsDataset(torchaudio.datasets.SPEECHCOMMANDS):
  def __init__(self,transform,root = "sample_data"):
    self.transform = transform
    super().__init__(root)

  # Override 
  def __getitem__(self,n):
    waveform, sample_rate, label, speaker_id, utterance_number = super().__getitem__(n)
    transformed_waveform = self.transform(waveform)
    return (transformed_waveform, sample_rate, label, speaker_id, utterance_number)


speech_commands_dataset = customSpeechCommandsDataset(transform = torch.nn.Sequential(PadWaveform(16000)))

Теперь можно добавлять дополнительные трансформации. Например уменьшить частоту дискретизации (sample_rate) что бы данные занимали меньше места.

Для этого в модуле:
[torchaudio.transforms](https://pytorch.org/audio/stable/transforms.html#resample)  уже есть готовая трансформация

In [ ]:
from torchaudio.transforms import Resample

speech_commands_dataset = customSpeechCommandsDataset(transform = torch.nn.Sequential(
    Resample(16000,8000),
    PadWaveform(8000))
)

### Визуализируем данные

Датасет в архиве занимает > 2Gb и это далеко не предел. Поэтому работать с ним будем по частям. 

Для этой задачи в pytorch используется класс Dataloader. Одной из его функций является пакетная(batch) загрузка данных. Особенно она будет полезна при обучении. 

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy

data_loader = torch.utils.data.DataLoader(speech_commands_dataset, batch_size=512, shuffle=True)

writer = SummaryWriter(comment = "commands")

for i, batch in enumerate(data_loader):
  waveforms, sample_rates, labels, speaker_ids, utterance_numbers = batch
  print(waveforms.shape)
  print(labels)
  # Данные преобразовались в тензоры
  # Убираем 1-е измерение оставшееся от канала
  writer.add_embedding(torch.squeeze(waveforms), metadata=labels )
  break
writer.close()

##Запустим Tensorboard

In [ ]:
reinit_tensorboard(False)

### Надо ли нормализовать эти данные?

Загрузим значения 2-х произвольных признаков в Tensorboard b проверим.

In [ ]:
writer = SummaryWriter(comment = "commands")
for i, batch in enumerate(data_loader):
  waveforms, sample_rates, labels, speaker_ids, utterance_numbers = batch
  writer.add_histogram("waves" ,torch.squeeze(waveforms)[:,100])
  writer.add_histogram("waves" ,torch.squeeze(waveforms)[:,200])
  break
writer.close()

Как видно из гистограммы, данные уже центрированны вкруг нуля и имеют один масшаб. Отчасти это связанно с тем что что они имеют одну и ту же природу, отчасти с форматом хранения звука. 

### Обучение

Для обучения потребуются метки. Попутно избавимся от лишнего. Создадим очередную трансформацию.


In [ ]:
class ClassName2Num(torch.nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, waveform):
    if waveform.shape[1] < self.desired_size:
      diff = self.desired_size - waveform.shape[1]
      pad_left = diff // 2
      pad_right = diff - pad_left
      return torch.nn.functional.pad(waveform,[pad_left, pad_right])
    else:
      return waveform

class customSpeechCommandsDatasetFinal(customSpeechCommandsDataset):
  def __init__(self,transform = torch.nn.Sequential(),root = "sample_data"):
    super().__init__(transform,root)
    self.labels = self.get_labels()

  def get_labels(self):
    labels = set()
    for i in range(len(self)):
      item = super(customSpeechCommandsDataset,self).__getitem__(i)
      labels.add(item[2])
    return sorted(list(labels)) 

  # Override 
  def __getitem__(self,n):
    waveform, sample_rate, label, speaker_id, utterance_number = super().__getitem__(n)
    return (waveform[0],self.labels.index(label))

speech_commands_dataset = customSpeechCommandsDatasetFinal(transform = torch.nn.Sequential(
    Resample(16000,8000),
    PadWaveform(8000))
)


In [ ]:
print("Classes",speech_commands_dataset.labels)
print("Classes num",len(speech_commands_dataset.labels))

wave, cls_num = speech_commands_dataset[0]
print(wave.shape)


Разделим данные на обучающую и валидационную выборки

In [ ]:
total_len = len(speech_commands_dataset )
print("Total length",total_len)
val_len = int(total_len*0.1)
train_set, val_set = torch.utils.data.random_split(speech_commands_dataset, [total_len - val_len, val_len])

In [ ]:
import numpy
from sklearn import metrics
from sklearn.linear_model import SGDClassifier

def validate(model):
  data_loader = torch.utils.data.DataLoader(val_set, batch_size=1000, shuffle=False)
  accuracy = []
  for batch in data_loader:
    waveforms, class_nums  = batch 
    y_pred = model.predict(waveforms)
    accuracy.append(metrics.accuracy_score(class_nums, y_pred))
  print("Accuracy:",numpy.array(accuracy).mean())

model = SGDClassifier(loss='log')  

data_loader = torch.utils.data.DataLoader(train_set, batch_size=20000, shuffle=True)
for batch in data_loader:
  waveforms, class_nums  = batch
  model.partial_fit(waveforms, class_nums,range(35))
  validate(model)

Точность низкая. Для работы с этими данными нужна глубокая модель.
С ее помощью можно получить точность >85%:

[Speech Command Recognition with torchaudio](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/d87597d0062580c9ec699193e951e3f4/speech_command_recognition_with_torchaudio.ipynb#scrollTo=tl9K6deU4S10)



## Место для рассказа про то чем строка в таблице принципиально отличается от аудиозаписи.


....

Вероятно тут стоит добавить несколько картинок.

....


* 1D - Таблица (столбцы не упорядоченны)
* 2D - Аудио (данные упорядоченны по времени)
* 3D - Монохромные изображения
* 4D - Цветные изображения, монохромные 3-х мерные изображения (МРТ)
* 5D - Видео, Воксельные изображения
* 6D - 3-мерное видео 

#Работа с изображениями

Загрузим датасет CIFAR-10. Он состоит из 60000 цветных изображений размером 32x32. На картинках объекты 10 классов.


 В отличие от torchaudio пакет torchvision при помощи которого загружается датасет входит в число предустановленных в colab.

Датасеты из torcvision изначально поддерживают механизм transforms - нам не придется добавлять их вручную.

Равно как и разбивку на тестовое и проверочные подмножества.

In [ ]:
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

trainset = datasets.CIFAR10("content", train=True,  download=True)
valset = datasets.CIFAR10("content", train = False, download=True)

Выведем несколько картинок вместе с метками. Tensorboard имееи метод для вывода кртинок:
[torchvision.utils.make_grid](https://pytorch.org/docs/stable/torchvision/utils.html)

Однако он не поддерживает метки.

https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjcxcStpNDtAhWllYsKHa7XDLoQFjAAegQIBBAC&url=https%3A%2F%2Fdiscuss.pytorch.org%2Ft%2Fadd-label-captions-to-make-grid%2F42863&usg=AOvVaw19bkv0_Q8VQxD7WBZ3pFR_

Поэтому воспользуемся matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
plt.rcParams["figure.figsize"] = (20,10)

# Загрузим названия классов. Исключительно для наглядности, для обучения модели они не нужны.
with open("content/cifar-10-batches-py/batches.meta",'rb') as infile:
  cifar_meta = pickle.load(infile)
labels = cifar_meta['label_names']

for j in range(10):
  image, class_num = trainset[j]
  plt.subplot(1, 10 ,j+1)
  plt.imshow(image)
  plt.axis('off')  
  plt.title(labels[class_num])


Посмотрим в каком виде храниться картинка в памяти

In [ ]:
trainset[0]

Оказывается в формате [PIL](https://pillow.readthedocs.io/en/stable/reference/Image.html)

Что бы обучать модель нам придется преобразовать их в тензоры. 
Используем для этого transforms и Dataloder.

Выведем размеры получившихся тензоров:

In [ ]:
from torch.utils.data import DataLoader
valset.transform = transforms.Compose([ transforms.ToTensor(),  ]) # PIL Image to Pytorch tensor
val_dataloder = DataLoader(valset, batch_size=8, shuffle=False)

for batch in train_dataloder:
  images, class_nums = batch
  print(len(batch))
  print("Images: ",images.shape)
  print("Class nums: ",class_nums.shape)
  break

Разберемся с размерностями:
на каждой итерации dataloader возвращает кортеж из двух элементов.
Первый это изображения, второй метки классов.

Количество элементов в каждом равно batch_size (8)

Изображение:
3 - C, каналы (В отличие от PIL и OpenCV они идут сначала)
32 - H, высота
32 - W, ширина 

Метки:
числа от 0 до 9 по количеству классов:

### Создадим модель - заглушку. 

Она не будет ничего предсказывать, только возвращать случайный номер класса.

В методе fit данные просто запоминаются. Этот фрагмент кода можно будет использовать при выполнении практического задания.


In [ ]:
import torch

class FakeModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.train_data = None
    self.train_labels = None

  def fit(self,x,y):
    self.train_data = torch.vstack((self.train_data,x)) if self.train_data != None else x
    self.train_labels = torch.hstack((self.train_labels,y)) if self.train_labels != None else y
   
  def forward(self,x):
    class_count = torch.unique(self.train_labels).shape[0]
    class_num = torch.randint(low = 0, high = class_count-1, size = (x.shape[0],)) 
    return class_num

Запустим процесс обучения

In [ ]:
trainset.transform = transforms.Compose([ transforms.ToTensor(),  ]) # PIL Image to Pytorch tensor
train_dataloder = DataLoader(trainset, batch_size=1024, shuffle=True)

model = FakeModel()

for img_batch, labels_batch in train_dataloder:
  model.fit(img_batch, labels_batch)

Проверим работу модели на нескольких изображениях из тестового набора данных

In [ ]:
img_batch, class_num_batch = next(iter(val_dataloder))
predicted_cls_nums = model(img_batch)

for i, predicted_cls_num in enumerate(predicted_cls_nums):
  img = img_batch[i].permute(1,2,0).numpy()*255  
  plt.subplot(1, len(predicted_cls_nums),i+1)
  plt.imshow(img.astype(int))
  plt.axis('off')
  plt.title(labels[int(predicted_cls_num)])

Посчитаем точность

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = []
for img_batch, labels_batch in val_dataloder:
  predicted = model(img_batch)
  batch_accuracy = accuracy_score(labels_batch, predicted)
  accuracy.append(batch_accuracy)

print("Accuracy",torch.tensor(accuracy).mean())


Будем повышать точность. Заменим заглушку в методе predict реальным алгоритмом. Используем метод:

K- Nearest Neighbor

https://colab.research.google.com/drive/1_5tGxAoxrWulPmwK2Ht9BHGsS-EpxVo0?usp=sharing




### 1.5.2Метод ближайшего соседа

<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/v1/knn.png" >

<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/intro-25.jpg" >
<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/intro-26.jpg" >
<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/intro-27.jpg" >
<img src ="https://edunet.kea.su/EduNet/source/L1_Intro/img/intro-28.jpg" >

Метод K- ближайших соседей на CIFAR10. 
Если рассматрить его по шагам, то нужно альтернативное задание на практическую часть семинара. 

# Практическая часть:

https://colab.research.google.com/drive/1EPP7XSydB_k-g3h73He67k7mH9pPqUFS?usp=sharing

#Блок про локальную настройку
...